<a href="https://colab.research.google.com/github/ManuelMC2203/TrabajoBD/blob/main/Proyecto_3_Clasificaci%C3%B3n_de_Texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto clasificación de datos de twitter

https://www.kaggle.com/datasets/aadyasingh55/twitter-emotion-classification-dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Download data with gdown
!gdown --id 1EguY8W_h17XnMeBd9AVlzPZj9Q6D_IFZ

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1EguY8W_h17XnMeBd9AVlzPZj9Q6D_IFZ
To: /content/train-00000-of-00001.parquet
100% 26.9M/26.9M [00:00<00:00, 70.1MB/s]


In [8]:
# Load parquet data using pandas
import pandas as pd

df = pd.read_parquet('train-00000-of-00001.parquet')

In [9]:
# Explore data
print("\n Some data: \n", df.head())

print("\n Data shape: \n", df.shape)

# count number of samples per class
print("\n Number of samples per class: \n", df['label'].value_counts())


 Some data: 
                                                 text  label
0  i feel awful about it too because it s my job ...      0
1                              im alone i feel awful      0
2  ive probably mentioned this before but i reall...      1
3           i was feeling a little low few days back      0
4  i beleive that i am much more sensitive to oth...      2

 Data shape: 
 (416809, 2)

 Number of samples per class: 
 label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64


In [5]:
# Use NLTK to remove stop words

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# List stop words for english
stop_words = set(stopwords.words('english'))
print(stop_words)

{'and', 'those', 'do', 'by', 'off', 'after', 'wouldn', 'hadn', 'them', 'y', "won't", 'than', 'why', 'both', 'if', 'are', 'should', 'him', 'on', 'was', 'these', 'an', 'below', 'nor', "shouldn't", 'they', 'out', 'their', 'here', 'to', 'ma', 'her', "should've", 'in', 'couldn', 'but', 'under', 'down', 'then', 'such', 'as', 'with', 'over', 're', 'theirs', 'just', 'its', 'you', 'own', 'from', 'yours', 'too', 've', 'i', "haven't", 'no', "isn't", "doesn't", 'into', 'had', 'yourself', 'what', 'for', 'when', 'himself', 'more', "don't", 'who', 'won', 'before', "that'll", 'she', 'few', "mightn't", 'o', 'haven', 'shan', 'am', 'our', 'each', 'hers', 'itself', 'mustn', "couldn't", 'yourselves', 'has', "it's", 'because', 'can', 'this', 'there', 'during', 'did', 'isn', 'does', 'will', "you'll", 'same', 'ourselves', 'now', "weren't", 'my', 'wasn', 'between', "you've", 'having', 'not', 'shouldn', 'be', 'of', 'further', 'doesn', 'once', 'a', 'or', "hadn't", 'that', "needn't", 'been', 'herself', 'your', "m

In [11]:
# example code to show how stop words filtering works

sentence = df['text'][3]

word_tokens = word_tokenize(sentence)

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

print(word_tokens)
print(filtered_sentence)

['i', 'was', 'feeling', 'a', 'little', 'low', 'few', 'days', 'back']
['feeling', 'little', 'low', 'days', 'back']


In [12]:
# function to filter stop words

def filter_stop_words(text):
  stop_words = set(stopwords.words('english'))

  word_tokens = word_tokenize(text)

  filtered_text = [w for w in word_tokens if not w.lower() in stop_words]

  return ' '.join(filtered_text)

In [13]:
# Test filter_stop_words function
print(filter_stop_words(df['text'][3]))

feeling little low days back


In [14]:
# Add an additional column 'filtered_text' to df appying stop_words_function to
# column 'text'
df['filtered_text'] = df['text'].apply(filter_stop_words)

In [15]:
# Verify the result
df.head()

,text,label,filtered_text
0,i feel awful about it too because it s my job ...,0,feel awful job get position succeed happen
1,im alone i feel awful,0,im alone feel awful
2,ive probably mentioned this before but i reall...,1,ive probably mentioned really feel proud actua...
3,i was feeling a little low few days back,0,feeling little low days back
4,i beleive that i am much more sensitive to oth...,2,beleive much sensitive peoples feelings tend c...



**Observaciones**

* En este momento los datos son textos filtrados de largo variable y una etiqueta de clase. Para poder proceder a la clasificación vamos a representar los textos con un vector numérico.
* CountVectorizer de sklearn tiene opciones de lower case y remoción de stop words.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Process filtered_text with CountVectorizer limiting to 20000 words
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['filtered_text'])

In [17]:
# Display X shape
print(X.shape)

(416809, 1000)


In [18]:
# train and test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.5, random_state=42, stratify=df['label'])

# Proyecto

El proyecto tiene por objetivo desarrollar un clasificador que permita clasificar los textos maximizando el accuracy.

Es importante que observen que en el código de referencia se limitaron las cantidad de palabras a 5000 cuando el total de palabras es significativamente mayor (puede probar CountVectorizer sin limitar el número de palabras para obtener el número total).

## Opción 1:
Desarrollar la solución mediante técnicas clásicas de clasificación.

## Opición 2:
Implementar la solución usando redes neuronales.